In [2]:
import utils
import enumeration
import utils_func
import argparse
import torch
import gc

from torchvision.transforms import transforms


gc.collect()
torch.cuda.empty_cache()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_NAMES_DICT = {
    "trades": "Zhang2019Theoretically",
    "mart": "Wang2020Improving",
    "wang2023": "Wang2023Better_WRN-28-10",
    'SAT' : 'Engstrom2019Robustness',
    'wu2020' : 'Wu2020Adversarial',
}


In [3]:
# parse arguments
parser = argparse.ArgumentParser(description="Process some parameters.")
parser.add_argument("--wandb_logging", type=bool, default=False, help="log to wandb")
parser.add_argument("--batch_size", type=int, default=64, help="batch size")
parser.add_argument(
    "--dataset",
    type=str,
    default="cifar10",
    choices=["cifar10", "cifar100"],
    help="dataset",
)
parser.add_argument("--epsilon", type=float, default=8 / 255, help="epsilon")
parser.add_argument("--alpha", type=float, default=2 / 255, help="alpha")
parser.add_argument(
    "--sampling_iterations", type=int, default=20, help="sampling iterations"
)
parser.add_argument(
    "--architecture_name", type=str, default="WideResNet", help="architecture name"
)
parser.add_argument(
    "--from_robustbench", type=bool, default=True, help="load model from robustbench"
)
parser.add_argument(
    "--init",
    type=str,
    default="random",
    choices=["random", "pca", "gaussian", "informative", "informative+pca"],
    help="initialization for the generation",
)

parser.add_argument("--seed", type=int, default=0, help="random seed")
parser.add_argument("--data_dir", type=str, default="./data", help="data directory")
parser.add_argument(
    "--model_name",
    type=str,
    default="Zhang2019Theoretically",
    choices=[MODEL_NAMES_DICT.keys()],
    help="define the name of the model to be loaded",
)
parser.add_argument(
    "--model_source",
    type=str,
    default="robustbench",
    choices=["robustbench", "jem", "local"],
    help="the source from where the model is loaded",
)
parser.add_argument(
    "--model_dir",
    type=str,
    default="./models",
)
parser.add_argument(
    "--threat model",
    type=str,
    default="Linf",
    choices=["Linf", "L2"],
    help="threat model",
)

_StoreAction(option_strings=['--threat model'], dest='threat model', nargs=None, const=None, default='Linf', type=<class 'str'>, choices=['Linf', 'L2'], required=False, help='threat model', metavar=None)

In [5]:

try:
    args = parser.parse_args()
    wandb = args.wandb_logging
    batch_size = args.batch_size
    dataset = args.dataset
    epsilon = args.epsilon
    alpha = args.alpha
    sampling_iterations = args.sampling_iterations
    architecture_name = args.architecture_name
    from_robustbench = args.from_robustbench
    initialization = args.init
    data_dir = args.data_dir
    seed = args.seed
    model_name = args.model_name
    model_source = args.model_source
    model_dir = args.model_dir
    threat_model = args.threat_model

    print("Using arguments from parser")

except:
    print("Using default arguments")
    wandb = False
    batch_size = 64
    dataset = "cifar10"
    epsilon = 8 / 255
    alpha = 2 / 255
    sampling_iterations = 20
    architecture_name = "WideResNet"
    from_robustbench = True
    init = "random"
    data_dir = "./data"
    seed = 0
    model_name = "Wang2020Improving"
    source = 'robustbench'
    model_dir = "./models"
    threat_model = "Linf"

Using default arguments


# threat model is {threath model}



In [6]:
from utils import fix_seed

fix_seed(seed=seed)

In [7]:

transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

dataset_train, dataset_test = utils.load_clean_dataset(
    dataset=enumeration.BenchmarkDataset(dataset.lower()),
    n_examples=None,
    data_dir=data_dir,
    transform=transform,
)
train_dataloader = torch.utils.data.DataLoader(
    dataset_train, batch_size=batch_size, shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(
    dataset_test, batch_size=batch_size, shuffle=False
)

Files already downloaded and verified
Files already downloaded and verified


## Define and load the model

The default option is to load the model whose name is specified in the arguments 

In [8]:
from robustbench.utils import load_model
from torch import nn

from JEMPP.models.jem_models import F, CCF
from JEMPP.models.wideresnet import Wide_ResNet as WideResNet


ARCHITECTURE_DICT = {'wideresnet': WideResNet}

assert architecture_name.lower() in ARCHITECTURE_DICT.keys()

### Define the model to load 

In [9]:
MODELS = {}
for m_name in MODEL_NAMES_DICT.keys():
    MODELS[m_name] = None


In [10]:
for i, model_name_i in enumerate(MODEL_NAMES_DICT.keys()):
    
    if model_source == "robustbench":
        model = load_model(
            model_name=MODEL_NAMES_DICT[model_name_i],
            dataset=dataset,
            threat_model=threat_model,
            model_dir="./models",
        )
        model = nn.DataParallel(model, device_ids=[0])
        model.to(device)
    elif model_source == "jem":
        model_cls = F
        model = model_cls(
            enumeration.dataset_image_size[dataset],
            enumeration.dataset_num_classes[dataset],
            norm="batch",
            n_classes=enumeration.dataset_num_classes[dataset],
            model="wrn",
        )
        ckpt_dict = torch.load("./models/cifar10/Linf/sadajem5_95.5_9.4.pt")
        model.load_state_dict(ckpt_dict["model_state_dict"])
        model = nn.DataParallel(model, device_ids=[0, 1])
        model.to(device)
        replay_buffer = ckpt_dict["replay_buffer"]

    elif model_source == "local":
        model_path = model_dir + "/wandb_model/" + model_name + ".pt"
        # check missimg 2 factors in architecture instanciation
        model = (
            ARCHITECTURE_DICT[architecture_name]()
            .to(device)
            .load_state_dict(torch.load(model_path))
            .eval()
        )
    
    MODELS[model_name_i]=model.module
    del model
    print(f"Model {model_name_i} loaded")



Model trades loaded
Model mart loaded
Model wang2023 loaded
Model SAT loaded
Model wu2020 loaded


In the cell below there is the computation of the mean of the dataset for each category and then the estimate of the dataset as a GMM where each class is a gaussian distribution

In [11]:
utils_func.category_mean(
    dload_train=train_dataloader,
    n_classes=enumeration.dataset_num_classes[dataset],
    name_dataset=dataset,
    image_size=enumeration.dataset_image_size[dataset],
    n_channels=3,
    data_dir=data_dir,
)
BUFFER = utils.center_initialization(
    dataset=dataset, n_classes=enumeration.dataset_num_classes[dataset]
)

REPLAY_BUFFER = []

> Mean and Covariance Already Computed, skipping


## Evaluation of the performance of the model for each initialization

The evaluation focuses on establishing the best way of initializing the generation.
The fixed parameters are :
- generated samples : 12 per class
- labels to generate : a subset of 10 labels 
- seed: defined as argument 
- the norm considered in the perturbation: Linf 
- Loss for the perturbation : energy_xy
- Dataset
- optimizer: SGLD vs Heun


In [12]:
num_samples = 5
label_to_plot = [i for i in range(10)]
label_to_plot = [[i for i in label_to_plot[:len(label_to_plot) // 2]], [i for i in label_to_plot[len(label_to_plot) // 2:]]]
#print(label_to_plot)
loss = 'reference_energy_xy'
sgld = 'SGLD'


## the init is `pca`

In [11]:
NUM_SAMPLES = 
LABEL_TO_PLOT = list(range(1))
#LABEL_TO_PLOT = [LABEL_TO_PLOT[:len(LABEL_TO_PLOT) // 2], LABEL_TO_PLOT[len(LABEL_TO_PLOT) // 2:]]
print(LABEL_TO_PLOT)
N_STEPS = 50



num_classes = 10
pca_ENERGY_DICT_xy_dist = {
    f"{i}_{j}": [] for j in range(num_classes) for i in range(num_classes)
}
size_plots = 20

print("PLOT of THE PCA INIT")

for labels in LABEL_TO_PLOT:
    init, img = [], []
    for name, model in MODELS.items():
        print(name)

        fix_seed(seed=seed)
        x0, imgs, energy = utils.new_samples(
            model=model,
            label=labels,
            batch_size=NUM_SAMPLES,
            step_size=0.5,
            n_steps=N_STEPS,
            inizialization="pca",
            n_steps_sampling=1,
            algo=sgld,
            loss=loss,
            replay_buffer=REPLAY_BUFFER,
            buffer=BUFFER,
            
            alpha=1,
            variance=0.95,
            noise_variance=0.001,
            compute_energy_model=True,
        )
        init.append(x0)
        img.append(imgs)

    init_cat = torch.cat(init, dim=0)

    img_cat = torch.cat(img, dim=0)

    utils_func.plot_imgs_generated(
        img_cat,
        inizialization="init: pca, loss: energy_xy, No HM, No momentum",
        rows=len(MODEL_NAMES_DICT.keys()),
        size=(size_plots, size_plots),
        path="./data/imgs.pdf",
        name="imgs: ",
    )
    for key, value in energy.items():
        if len(value) > 0:
            pca_ENERGY_DICT_xy_dist[key] = value


pca_ENERGY_DICT={
    f"{i}_{j}": [] for j in range(num_classes) for i in range(num_classes)
}
for key, value in pca_ENERGY_DICT_xy_dist.items():
    for sample in range(NUM_SAMPLES):
        flag_list = [value[i+sample] for i in range(0, len(value), NUM_SAMPLES)]
        pca_ENERGY_DICT[key].append(flag_list)


[0]
PLOT of THE PCA INIT
SAT


/home/mary/miniconda3/envs/robustbench/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED